In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
from numpy.linalg import norm
from sklearn.decomposition import PCA
from PIL import Image as PILImage


# Step 1: Load Pre-trained CNN
base_model = ResNet50(weights='imagenet')
# We'll use the output of the layer just before the final dense layer (usually named 'avg_pool' for ResNet)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

# Step 2: Preprocess the Image
def preprocess_image_pillow(img_path):
    img = PILImage.open(img_path)
    img = img.resize((224, 224))  # Resize image to 224x224
    img_array = np.array(img)
    
    # If the image has an alpha channel, we should remove it
    if img_array.shape[2] == 4:
        img_array = img_array[:, :, :3]

    # Convert the image array to float and rescale it
    img_array = img_array.astype(np.float32)

    # Preprocess the image for the model (ResNet in this case)
    # This step depends on the pre-trained model's expected input
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Ensure you use the correct preprocess_input function for your model

    return img_array

# Step 3: Extract Features
def extract_features(img_path, model):
    preprocessed_image = preprocess_image_pillow(img_path)
    features = model.predict(preprocessed_image)
    flattened_features = features.flatten()  # Flatten the features to a 1D array
    return flattened_features

# Step 4: Normalize Features
def normalize_features(features):
    # Normalize feature vector (L2 norm)
    normalized_features = features / np.linalg.norm(features)
    return normalized_features

# Step 5: Reduce Dimensionality
def reduce_dimensionality(features, n_components=300):
    # Initialize PCA object to reduce to n_components dimensions
    pca = PCA(n_components=n_components)
    features = np.array(features)
    pca.fit(features)
    reduced_features = pca.transform(features)
    return reduced_features



# Function to calculate cosine similarity
def cosine_similarity(vec_a, vec_b):
    similarity = np.dot(vec_a, vec_b) / (norm(vec_a) * norm(vec_b))
    return similarity



# Sample Usage
# Assuming 'image_path' is the path to the image file
mouse1 = r'C:\Users\ayhan\Desktop\Smart-Shopper-Recommendation-Engine\mouse1.jpg'
mouse2 = r'C:\Users\ayhan\Desktop\Smart-Shopper-Recommendation-Engine\mouse2.jpg'
klavye = r'C:\Users\ayhan\Desktop\Smart-Shopper-Recommendation-Engine\klavye.jpg'
laptop = r'C:\Users\ayhan\Desktop\Smart-Shopper-Recommendation-Engine\laptop.jpg'
nutella = r'C:\Users\ayhan\Desktop\Smart-Shopper-Recommendation-Engine\nutella.jpg'

features1 = extract_features(mouse1, model)
normalized_features1 = normalize_features(features1)
features2 = extract_features(mouse2, model)
normalized_features2 = normalize_features(features2)
features3 = extract_features(klavye, model)
normalized_features3 = normalize_features(features3)
features4 = extract_features(laptop, model)
normalized_features4 = normalize_features(features4)
features5 = extract_features(nutella, model)
normalized_features5 = normalize_features(features5)




1/1 [==============================] - 0s 96ms/step


In [9]:
### PERFORM SIMILARIY SEARCH BTW IMAGES

similarity_with_similar = cosine_similarity(normalized_features1, normalized_features2)
similarity_with_irrelevant = cosine_similarity(normalized_features1, normalized_features3)
similarity_with_irrelevant2 = cosine_similarity(normalized_features1, normalized_features4)

print("Similarity with similar image:", similarity_with_similar)
print("Similarity with irrelevant image:", similarity_with_irrelevant)
print("Similarity with nutella image:", cosine_similarity(normalized_features1, normalized_features5))

Similarity with similar image: 0.7657989
Similarity with irrelevant image: 0.34152448
Similarity with nutella image: 0.36515915
